# 실행 순서 
------
+ ## 1. DataDownload 
+ ## 2. EDA 
 ### 1. train, test pixel 값의 EDA
 ### 2. train ,test class 별 데이터 갯수 

##3. DNN 구성.

##4. metric 측정 
- (tensorboard accuracy, loss step별로 측정)

##5. analysis 
- 각 층별 weights, bias 평균의 변화 측정 (step 별)*이탤릭체 텍스트*
------

### 1. EMNIST DataDownLOAD 

In [0]:
!pip install emnist

In [0]:
import emnist
import matplotlib.pyplot as plt 
import numpy as np 

In [0]:
train_images, train_labels = emnist.extract_training_samples('letters')
test_images, test_labels = emnist.extract_test_samples('letters')

In [0]:
# show sample data 
print(train_images.shape)
plt.imshow(train_images[0])
plt.show()
plt.imshow(train_images[1])
plt.show()

#### Image EDA 

In [0]:
# Train Image 
n_train = len(train_images)
seq_length = train_images.shape[1:3]

train_min = train_images.reshape([-1,784]).min(axis=1)
train_max = train_images.reshape([-1,784]).max(axis=1)

print('# train : {}'.format(n_train))
print('seq_length : {}'.format(seq_length))

# 아래 그림으로 보아 모든 Test 이미지가 최소값은 0, 최대값은 255을 가지고 있음. 
plt.scatter(train_min, train_max)

In [0]:
# Test Image 
n_test = len(test_images)
seq_length = test_images.shape[1:3]

test_min = test_images.reshape([-1,784]).min(axis=1)
test_max = test_images.reshape([-1,784]).max(axis=1)

print('# train : {}'.format(n_test))
print('seq_length : {}'.format(seq_length))

# 아래 그림으로 보아 모든 Test 이미지가 최소값은 0, 최대값은 255을 가지고 있음. 
plt.scatter(test_min, test_max)

#### Label EDA 

In [0]:
print('train label data shape : {}'.format(train_labels.shape))
print('test label data shape : {}'.format(test_labels.shape))

print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

train_labels = train_labels - 1 
test_labels = test_labels - 1 

### Normalize and Normalization checking  

In [0]:
np.unique([train_labels],return_counts=True)

# Normalization
train_images = train_images.reshape(-1,784)/ 255.
test_images = test_images.reshape(-1,784) / 255.

# Check Normalization 
def chk_norm(input_data, min_value=0, max_value=1):
    """
    input_data : Ndarray, 
    min_value : int, input_data의 모든 element는 min_value 이상의 값을 가져야함 
    max_value : int, input_data의 모든 element는 max_value 이하의 값을 가져야함 

    description: input_data의 모든 element가 특정 범위(min_value ~ max_value)에 있는지 확인합니다.
    """
    input_data = np.asarray(input_data)
    assert np.all(input_data <= max_value) & np.all(input_data >= min_value), 'normalize가 잘못 되었습니다.'


chk_norm(train_images)    
chk_norm(test_images)

## Input layer

In [0]:
import tensorflow as tf 
num_inputs = 28*28 # MNIST Input size
num_outputs = 26 # The number of Label : 26

tf.reset_default_graph()

# Data를 받아오는 placeholder
x = tf.placeholder(tf.float32, shape=[None, num_inputs],name='x')

#labels shape : [1,3,7,7,4,..]
labels_cls = tf.placeholder(tf.int32, shape=[None], name='labels') 

# scalar 을 onehot-vector 형태로 변환합니다.
labels = tf.one_hot(labels_cls, depth=num_outputs)

learning_rate = tf.placeholder_with_default(0.1,shape=(),
                                            name='learning_rate')

## Model

In [0]:
num_hidden1 = 64
num_hidden2 = 128
num_hidden3 = 256

# Weight 초기화
with tf.variable_scope('hidden1'):
    w1 = tf.Variable(tf.random.normal([num_inputs,num_hidden1],
                                      stddev=np.sqrt(2/num_inputs)),
                     name='weight')
    b1 = tf.Variable(tf.zeros([num_hidden1])
                     ,name='bias')
    
    z1 = tf.matmul(x, w1) + b1
    a1 = tf.nn.relu(z1)

with tf.variable_scope('hidden2'):
    w2 = tf.Variable(tf.random.normal([num_hidden1,num_hidden2],
                                      stddev=np.sqrt(2/num_hidden1)),
                     name='weight')
    b2 = tf.Variable(tf.zeros([num_hidden2])
                     ,name='bias')

    z2 = tf.matmul(a1, w2) + b2
    a2 = tf.nn.relu(z2)

    
with tf.variable_scope('hidden2'):
    w3 = tf.Variable(tf.random.normal([num_hidden2,num_hidden3],
                                      stddev=np.sqrt(2/num_hidden2)),
                     name='weight')
    b3 = tf.Variable(tf.zeros([num_hidden3])
                     ,name='bias')

    z3 = tf.matmul(a2, w3) + b3
    a3 = tf.nn.relu(z3)
    

with tf.variable_scope('output'):
    wo = tf.Variable(tf.random.normal([num_hidden3,num_outputs],
                                      stddev=np.sqrt(2/num_hidden3)),
                     name='weight')
    bo = tf.Variable(tf.zeros([num_outputs])
                     ,name='bias')

    logits = tf.matmul(a3, wo) + bo
logits = tf.identity(logits, name='logits')



## Loss Function 

In [0]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
mean_loss = tf.reduce_mean(loss)

## Gradient 

In [0]:
vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
grad_vars = tf.gradients(mean_loss, vars)
tf.add_to_collection('GRADIENTS', grad_vars)

## Train op

In [0]:
## low api 을 이용해 Gradient Descent 구현 
# assign_ops = []
# grad_var = tf.get_collection('GRADIENTS')
# print(grad_var)
# for var_, grad in  zip(vars, grad_var):
#     print(var_, grad[0])
#     assign_op = tf.assign_sub(var_, grad[0]*learning_rate)
#     assign_ops.append(assign_op)
# train_ops =tf.group(assign_ops)

In [0]:
## high level api을 이용해 Gradient Descent 구현
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(mean_loss)

## next batch function

In [0]:
# 복원 추출 합니다. 
import random
def next_batch(xs, ys, batch_size):
    indices = random.sample(range(len(ys)), batch_size)
    return xs[indices], ys[indices]


## Metric

In [0]:
logits_cls = tf.argmax(logits, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(logits_cls, labels_cls), 
                     dtype=tf.float32))

## Session Open

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## training


In [0]:
max_iter = 20000
batch_size = 120 
mean_gradients=[]
accum_train_acc = [] 
accum_train_loss = [] 
accum_test_acc = [] 
accum_test_loss = [] 

for i in range(max_iter):
    # random select input data, labels
    batch_xs, batch_ys = next_batch(train_images, train_labels, batch_size)
    # Training 
    train_acc, train_loss, _ = sess.run([acc, mean_loss, train_op], 
                                        feed_dict={x: batch_xs,
                                                   labels_cls: batch_ys,
                                                  learning_rate:0.1})
    # Evaluating
    if i % 100 == 0:
        test_acc, test_loss = sess.run([acc, mean_loss], 
                                    feed_dict={x:test_images,
                                               labels_cls: test_labels})
        print('step : {} train_acc : {:.4f} train_loss : {:.4f} validation acc : {:.4f} validation loss : {:.4f}'.\
              format(i, train_acc, train_loss, test_acc, test_loss))
        
        # list 에 추출함.
        accum_train_acc.append(train_acc)
        accum_train_loss.append(train_loss)
        accum_test_acc.append(test_acc)        
        accum_test_loss.append(test_loss)        

## Visualization

In [0]:
plt.plot(range(0,20000,100), accum_train_acc, label='train')
plt.plot(range(0,20000,100), accum_test_acc, label='test')
plt.title('Accuracy')
plt.legend()


In [0]:
plt.plot(range(0,20000,100), accum_train_loss, label='train')
plt.plot(range(0,20000,100), accum_test_loss, label='test')
plt.title('Loss')
plt.legend()
